<a href="https://colab.research.google.com/github/simon-m-mudd/smm_teaching_notebooks/blob/master/transience_from_topography/Common_analyses_to_check_for_transience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common analyses that you can use to check for landscape transience

*This lesson made by Simon M Mudd and last updated 18/09/2023*

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

The assumption for these notebooks is that you are working in a notable container.

So these steps are turned into comments using a `#` symbol before each line.

If you want to work on google colab you will need to delete the `#` symbols

In [ ]:
#!pip install lsdviztools &> /dev/null

In [ ]:
#!pip install -q condacolab
#import condacolab
#condacolab.install()

In [ ]:
#!mamba install -y gdal proj lsdtopotools &> /dev/null

## Part 1: Get data

**First step, copy your Opentopography API key into a text file called "my_OT_api_key.txt" and make sure it is in the same file system as this notebook.**

If you don't know how to do that got to the `basic_topography` section and read lesson 1.

Make sure we are using the latest version of `lsdviztools`

In [ ]:
!pip install lsdviztools --upgrade

Double check that the version is correct. For this lesson we need version 0.4.7 or above.

In [ ]:
import lsdviztools
lsdviztools.__version__

We also need to import some stuff

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

**Next step: make sure the filename of your API key matches the one below.** Filenames are case sensitive. So if your file is "my_OT_API_key.txt" this will not work in the code block below (because api needs to be lower case).

**Then update the latitude and longitude of the lower left and upper right corner of your study area. You can get a latitude and longitude by right clicking on google maps.

In [ ]:
lower_left = [37.87521641344407, 15.628317625157045]
upper_right = [38.30554438686832, 16.210748067627133]

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "DEM"
source_name = "COP30"


My_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = lower_left,
                        upper_right_coordinates = upper_right,
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
My_DEM.print_parameters()
My_DEM.download_pythonic()

## Part 2: Get a hillshade

We are going to get a hillshade since this is used in all of the plotting routines in `lsdviztools`.

*You only need to execute this code, you don't need to change anything!*

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_basin_raster" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This prints the image. It will also save a png file.

In [ ]:
DataDirectory = "./"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=True,
                                 size_format="geomorphology",dpi=500,
                                 out_fname_prefix = "hillshade_plot")

print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

## Part 3 Make slope and curvature maps

In [ ]:
# IMPORTANT all the parameter values must be passed as strings.
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"surface_fitting_radius" : "60",
                    "print_slope" : "true",
                    "print_curvature" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
%%capture
Base_file = r_prefix

### Plot the topographic gradient
Drape_prefix = r_prefix+"_SLOPE"
this_slope_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="inferno",
                                       cbar_label = "Gradient (m/m)",
                                       save_fig=True, size_format="geomorphology",dpi=600,
                                       out_fname_prefix = "gradient_plot")

In [ ]:
print(this_slope_img)
from IPython.display import display, Image
display(Image(filename=this_slope_img, width=800))

In [ ]:
%%capture
Base_file = r_prefix

### Plot the curvature
Drape_prefix = r_prefix+"_CURV"
this_curv_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="RdBu",
                                  cbar_label = "Curvature (1/m)",
                                  cbar_loc = "top",
                                  save_fig=True, size_format="ESURF",dpi=600,
                                  out_fname_prefix = "curvature_plot")

In [ ]:
print(this_curv_img)
from IPython.display import display, Image
display(Image(filename=this_slope_img, width=800))

## Part 4: Select basins

In [ ]:
# Import pandas library
import pandas as pd

data = [ [37.94234239772101, 15.790076672680563],
         [37.93291942111813, 15.792125654538383],
         [38.06143176957989, 16.060604039819015],
         [38.094672987877786, 15.677112531901733],
         [38.20455853467961, 15.67642588617276],
         [38.12546843046537, 16.1230887795859]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_basin_raster" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "print_chi_data_maps" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
# uncomment this for debugging
#import lsdviztools.lsdmapwrappers as lsdmw
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=600, save_fig = True,
                             out_fname_prefix = "basins_plot")

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

## Part 5: Plot channel profiles

In [ ]:
import pandas as pd
channel_file_name = r_prefix+"_chi_data_map.csv"
df = pd.read_csv(channel_file_name)
df.head()

In [ ]:
basin = 4

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.chi,df_single_basin.elevation,c=df_single_basin.flow_distance)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

chi_profile_name = "chi_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(chi_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.flow_distance,df_single_basin.elevation,c=df_single_basin.chi)
plt.xlabel(r"$\flow distance$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

channel_profile_name = "channel_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(channel_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

## A swath example

In [ ]:
# Import pandas library
import pandas as pd

data = [ [-29.621021116107862, 28.57147034542054],
         [-29.783077317434813, 28.281779758189604]
       ]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("swath1.csv",index=False)
df.head()

In [ ]:
## Get the basins and the channel profile
import lsdviztools.lsdmapwrappers as lsdmw
lsdtt_parameters = {"calculate_swath_profile" : "true",
                    "calculate_swath_along_line" : "true",
                    "swath_points_csv" : "swath1.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
### Plot the swath footprint
Base_file = r_prefix
Drape_prefix = Base_file+"_swathval"
DataDirectory = "./"
this_area_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="jet",
                                       cbar_label = "Swath distance (m)",
                                       cbar_loc = "bottom",
                                       save_fig=True, size_format="ESURF",dpi=600,
                                       out_fname_prefix = "swath_footprint_plot")

In [ ]:
print(this_area_img)
from IPython.display import display, Image
display(Image(filename=this_area_img, width=800))

In [ ]:
import lsdviztools.lsdplottingtools.lsdmap_swathplotting as lsdsp
import lsdviztools.lsdmapwrappers as lsdmw
DataDirectory="./"
swathname = r_prefix+"_swath.csv"
imname = lsdmw.SimpleSwath(DataDirectory,swathname, fig_format = "png", dpi = 500, aspect_ratio = 2)

In [ ]:
from IPython.display import display, Image
display(Image(filename=imname, width=800))